In [133]:
from pycorenlp import StanfordCoreNLP
import json

nlp = StanfordCoreNLP('http://localhost:9000')
dependency_parser = nlp.annotate

def get_ner(sentence):
    result = dependency_parser(sentence, properties={
        "outputFormat": "json",
        "annotators": "ner"})
    res = []
    for obj in result['sentences'][0]['entitymentions']:
        res.append((obj['text'], obj['ner']))
    return res

In [134]:
get_ner("you owe obama $4 in indonesia")

[('$4', 'MONEY'), ('indonesia', 'COUNTRY')]

In [135]:
import pandas as pd
from tqdm import  tqdm_notebook

all_tipe = []
def run(tipe):
    df = pd.read_csv("Results/add_noun/"+ tipe +".csv")
    sf = pd.DataFrame(columns = ['id', 'review', 'target', 'category', 'term', 'polarity'])
    count = 0
    for index in tqdm_notebook(range(len(df))):
        all_ner = []
        ner = get_ner(df['review'][index])
        if len(ner):
            for word, t in ner:
                if t != 'MONEY'and t != 'TITLE':
#                     print(t)
                    if t not in all_tipe:
                        all_tipe.append(t)
                    all_ner.append(word)
#                 if t == 'TITLE':
#                     print(df['review'][index], word)
#                     print(df['category'][index])
        
        terms = []
        if type(df['term'][index]) != float:
            terms = df['term'][index].split('|')
            
#         print()
#         print(terms_dp)
#         if len(all_ner):
        new_term = []
#         print(all_ner)
        for kata in terms:
            flag = True
            for n in all_ner:
                if kata in n.lower() and kata != '':
#                     print(df['review'][index])
#                     print(kata)
#                     print(terms)
#                     print(n)
                    flag = False
            if ('time' in kata or 'thing' in kata) and kata != '':
#                 print(kata)
                flag = False
            if flag:
                new_term.append(kata)
#             else:
#                 print('ooooooooooooooooooooooooooooooo')
#         else:
#             new_term = terms_dp
        
#         print(new_term)
        sf = sf.append({'id': df['id'][index], 
                    'review': df['review'][index],
                    'target': df['target'][index],
                    'category': df['category'][index],
                    'term': '|'.join(new_term),
                    'polarity': df['polarity'][index]
                   }, ignore_index=True)
    print(count)

    sf.to_csv("Results/Filter/"+ tipe +"-ner.csv")
    sf.to_excel("Results/Filter/"+ tipe +"-ner.xlsx")

In [129]:
# run('dp-partial')
ate = ['dp', 'lda', 'lda2vec', 'plsa']
sim = ['glove', 'w2v', 'fasttext', 'elmo', 'elmo_ori']

# ate = ['lda2vec']
# sim = ['elmo']

for a in ate:
#     for s in sim:
    run(a + '-partial')

HBox(children=(IntProgress(value=0, max=462), HTML(value='')))

0


HBox(children=(IntProgress(value=0, max=462), HTML(value='')))

0


HBox(children=(IntProgress(value=0, max=462), HTML(value='')))

0


HBox(children=(IntProgress(value=0, max=462), HTML(value='')))

0


In [108]:
all_tipe

['NUMBER',
 'TIME',
 'PERSON',
 'NATIONALITY',
 'DATE',
 'DURATION',
 'STATE_OR_PROVINCE',
 'COUNTRY',
 'ORDINAL',
 'SET']

In [107]:
sorted(list(dict.fromkeys(all_tipe)))

['COUNTRY',
 'DATE',
 'DURATION',
 'NATIONALITY',
 'NUMBER',
 'ORDINAL',
 'PERSON',
 'SET',
 'STATE_OR_PROVINCE',
 'TIME']

In [136]:
import string
def rem_punct(word):
    return word.translate(str.maketrans('', '', string.punctuation))

In [137]:
import pandas as pd
from tqdm import  tqdm_notebook

def run_price(tipe):
    df = pd.read_csv("Results/Categorization/"+ tipe +".csv")
    sf = pd.DataFrame(columns = ['id', 'review', 'target', 'label', 'predict', 'term', 'polarity'])
    count = 0
    for index in tqdm_notebook(range(len(df))):
        all_ner = []
        ner = get_ner(df['review'][index])
        if len(ner):
            for word, t in ner:
                if t == 'MONEY':
                    all_ner.append(word)
        terms_dp = []
        if type(df['term'][index]) != float:
            terms_dp = df['term'][index].split('|')
        
        predicts = []
        if type(df['predict'][index]) != float:
            predicts = df['predict'][index].split('|')
            
        new_term = []
        if len(all_ner):
            for t_dp in terms_dp:
                new_term.append(t_dp)
            for n in all_ner:
                new_term.append(n + '!-')
                predicts.append('PRICES')
                print(n)
        else:
            new_term = terms_dp
        
        sf = sf.append({'id': df['id'][index], 
                    'review': df['review'][index],
                    'target': df['target'][index],
                    'label': df['label'][index],
                    'predict': '|'.join(predicts),
                    'term': '|'.join(new_term),
                    'polarity': df['polarity'][index]
                   }, ignore_index=True)
    print(count)

    sf.to_csv("Results/Categorization/"+ tipe +"-money.csv")
    sf.to_excel("Results/Categorization/"+ tipe +"-money.xlsx")
    print('sksk')

In [138]:
# run_price('output')

In [139]:
ate = ['dp', 'lda', 'lda2vec', 'plsa']
sim = ['glove', 'w2v', 'fasttext', 'elmo', 'elmo_ori']

# ate = ['lda2vec']
# sim = ['elmo']

for a in ate:
    for s in sim:
        run_price(a + '-partial-ner-' + s)

HBox(children=(IntProgress(value=0, max=462), HTML(value='')))

$6
$20
$3
$8.00
$18
$3-6
$70
$30
$7
$24
$10
100 dollar
$*
$8.50
$20
$15
$60
$400.00
$11
$8
$10
$500
$500
$14
$55
$25
$
$2
$29
$500
$24
$60
0
sksk


HBox(children=(IntProgress(value=0, max=462), HTML(value='')))

$6
$20
$3
$8.00
$18
$3-6
$70
$30
$7
$24
$10
100 dollar
$*
$8.50
$20
$15
$60
$400.00
$11
$8
$10
$500
$500
$14
$55
$25
$
$2
$29
$500
$24
$60
0
sksk


HBox(children=(IntProgress(value=0, max=462), HTML(value='')))

...

$6
$20
$3
$8.00
$18
$3-6
$70
$30
$7
$24
$10
100 dollar
$*
$8.50
$20
$15
$60
$400.00
$11
$8
$10
$500
$500
$14
$55
$25
$
$2
$29
$500
$24
$60
0
sksk


HBox(children=(IntProgress(value=0, max=462), HTML(value='')))

$6
$20
$3
$8.00
$18
$3-6
$70
$30
$7
$24
$10
100 dollar
$*
$8.50
$20
$15
$60
$400.00
$11
$8
$10
$500
$500
$14
$55
$25
$
$2
$29
$500
$24
$60
0
sksk


HBox(children=(IntProgress(value=0, max=462), HTML(value='')))

$6
$20
$3
$8.00
$18
$3-6
$70
$30
$7
$24
$10
100 dollar
$*
$8.50
$20
$15
$60
$400.00
$11
$8
$10
$500
$500
$14
$55
$25
$
$2
$29
$500
$24
$60
0
sksk


In [12]:
'|'.join(['sd'])

'sd'